In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io.arff import loadarff
from sklearn.preprocessing import LabelEncoder
import glob
import logging

logging.basicConfig(level=logging.INFO, format="%(levelname)s - %(message)s")
logger = logging.getLogger()

In [2]:
def load_data(
    train_file_pattern: str, test_file_pattern: str
) -> tuple[list[pd.DataFrame], list[pd.DataFrame]]:
    """
    Load train and test data from ARFF files matching the given patterns.

    This function reads ARFF files for both training and test datasets,
    converts them to pandas DataFrames, and returns them as lists.

    Parameters:
    -----------
    train_file_pattern : str
        Glob pattern to match training data files.
    test_file_pattern : str
        Glob pattern to match test data files.

    Returns:
    --------
    tuple[list[pd.DataFrame], list[pd.DataFrame]]
        A tuple containing two lists:
        1. List of DataFrames for training data.
        2. List of DataFrames for test data.

    Note:
    -----
    The function uses scipy.io.arff.loadarff to read ARFF files and
    converts the data to pandas DataFrames. Files are sorted to ensure
    consistent ordering across runs.
    """
    train_dfs = []
    test_dfs = []
    for is_test, file_pattern in [
        (False, train_file_pattern),
        (True, test_file_pattern),
    ]:
        for file in sorted(glob.glob(file_pattern)):
            raw_data, meta = loadarff(file)
            logger.info(f"Loading {file}")
            df = pd.DataFrame(raw_data, columns=meta.names())
            if is_test:
                test_dfs.append(df)
            else:
                train_dfs.append(df)
    return train_dfs, test_dfs

In [3]:
train_dfs, test_dfs = load_data(
    "datasetsCBR/mushroom/mushroom.fold.*.train.arff",
    "datasetsCBR/mushroom/mushroom.fold.*.test.arff",
)

# Display the number of train and test dataframes
print(f"\nNumber of training dataframes: {len(train_dfs)}")
print(f"Number of test dataframes: {len(test_dfs)}")
print()
train_dfs[0].head()

INFO - Loading datasetsCBR/mushroom/mushroom.fold.000000.train.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000001.train.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000002.train.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000003.train.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000004.train.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000005.train.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000006.train.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000007.train.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000008.train.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000009.train.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000000.test.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000001.test.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000002.test.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.000003.test.arff
INFO - Loading datasetsCBR/mushroom/mushroom.fold.00


Number of training dataframes: 10
Number of test dataframes: 10



,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,b'x',b's',b'w',b't',b'l',b'f',b'c',b'b',b'k',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b'n',b'm',b'e'
1,b'k',b'y',b'e',b'f',b's',b'f',b'c',b'n',b'b',b't',...,b'p',b'w',b'p',b'w',b'o',b'e',b'w',b'v',b'd',b'p'
2,b'x',b'y',b'y',b't',b'a',b'f',b'c',b'b',b'n',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b's',b'm',b'e'
3,b'x',b'y',b'w',b't',b'p',b'f',b'c',b'n',b'k',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b'v',b'u',b'p'
4,b'x',b'y',b'n',b'f',b'f',b'f',b'c',b'n',b'b',b't',...,b'w',b'p',b'p',b'w',b'o',b'e',b'w',b'v',b'l',b'p'


In [4]:
for df in train_dfs + test_dfs:
    for col in df.columns:
        label_encoder = LabelEncoder()
        df[col] = label_encoder.fit_transform(df[col])

train_dfs[0].head()

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,5,2,8,1,3,1,0,0,4,0,...,7,7,0,2,1,4,2,2,3,0
1,3,3,2,0,7,1,0,1,0,1,...,6,7,0,2,1,0,7,4,0,1
2,5,3,9,1,0,1,0,0,5,0,...,7,7,0,2,1,4,2,3,3,0
3,5,3,8,1,6,1,0,1,4,0,...,7,7,0,2,1,4,2,4,5,1
4,5,3,4,0,2,1,0,1,0,1,...,7,6,0,2,1,0,7,4,2,1


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

results = []

for train_df, test_df in zip(train_dfs, test_dfs):
    decision_tree = DecisionTreeClassifier()
    decision_tree.fit(train_df.drop("class", axis=1), train_df["class"])
    preds = decision_tree.predict(test_df.drop("class", axis=1))
    results.append(accuracy_score(test_df["class"], preds))

print(f"Mean accuracy: {np.mean(results)}")

Mean accuracy: 1.0
